In [100]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [101]:
df = pd.read_csv('./data/sample.csv')
print(df.shape)
df.head()

(29236, 18)


,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2466149,2,2017-03-06 12:10:08,2017-03-06 12:20:21,2,2.03,1,N,140,236,1,9.0,0.0,0.5,1.96,0.0,0.3,11.76
1,3406713,1,2017-06-14 08:11:30,2017-06-14 08:14:38,1,0.40,1,N,43,142,1,4.5,0.0,0.5,1.05,0.0,0.3,6.35
2,7071298,2,2017-11-22 19:42:47,2017-11-22 19:49:49,3,1.04,1,N,151,238,1,6.5,1.0,0.5,0.00,0.0,0.3,8.30
3,3845741,2,2017-11-12 16:58:55,2017-11-12 17:09:24,1,1.79,1,N,239,236,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30
4,3672997,1,2017-06-14 22:01:39,2017-06-14 22:06:32,1,1.20,1,N,246,246,1,6.0,0.5,0.5,2.15,0.0,0.3,9.45


In [102]:
# train, test split
# set seed for reproducibility 

X_train, X_test, y_train, y_test = train_test_split(df, df.tip_amount,
                                                    test_size=0.1,
                                                    random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((26312, 18), (2924, 18), (26312,), (2924,))

In [103]:
df.columns

Index(['Unnamed: 0', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
       'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount',
       'tolls_amount', 'improvement_surcharge', 'total_amount'],
      dtype='object')

In [104]:
df.tpep_dropoff_datetime

0        2017-03-06 12:20:21
1        2017-06-14 08:14:38
2        2017-11-22 19:49:49
3        2017-11-12 17:09:24
4        2017-06-14 22:06:32
5        2017-06-23 09:37:23
6        2017-11-28 19:54:24
7        2017-06-17 18:16:56
8        2017-06-14 00:40:42
9        2017-06-29 14:41:42
10       2017-03-06 18:22:59
11       2017-11-20 23:13:46
12       2017-03-10 18:43:18
13       2017-03-11 11:19:11
14       2017-03-20 16:56:24
15       2017-11-02 22:23:54
16       2017-06-13 09:49:59
17       2017-11-26 07:46:58
18       2017-03-28 21:13:08
19       2017-03-15 20:01:03
20       2017-11-03 18:07:08
21       2017-11-01 21:37:46
22       2017-11-05 01:26:53
23       2017-06-25 21:27:55
24       2017-06-30 12:34:17
25       2017-03-29 14:03:15
26       2017-06-22 19:27:14
27       2017-03-08 20:30:23
28       2017-11-02 20:21:32
29       2017-03-04 00:09:25
                ...         
29206    2017-06-10 03:17:28
29207    2017-03-30 18:17:30
29208    2017-03-19 16:21:28
29209    2017-

### Time variable

In [105]:
def trip_time(df, start, end):
    """
    start: pickup time
    end: dropoff time
    
    Extracts time information from the two variables. 
    """
    df = df.copy()
    
    df['pickup'] = pd.to_datetime(df[start])
    df['dropoff'] = pd.to_datetime(df[end])
    
    df['trip_time'] = df['dropoff'] - df['pickup']
    
    df = df.drop(columns=[start, end])
    
    return df

In [106]:
X_train = trip_time(df, 'tpep_pickup_datetime', 'tpep_dropoff_datetime')
X_test = trip_time(df, 'tpep_pickup_datetime', 'tpep_dropoff_datetime')

### Categorical variables
- convert strings to numbers to capture monotonic relationship between label and target

In [107]:
# for one hot encoding
# a few variables have been coded as an int instead of a categorical variable
var_to_str = ['RatecodeID', 'store_and_fwd_flag', 'payment_type']

def to_str(train, test, var):
    train[var] = train[var].astype(str)
    test[var] = test[var].astype(str)
    
for var in var_to_str:
    to_str(X_train, X_test, var)

In [108]:
# check cardinality of non-numeric features we want to one-hot encode
X_train.describe(include='O').T.sort_values(by='unique')

,count,unique,top,freq
store_and_fwd_flag,29236,2,N,29082
payment_type,29236,4,1,19899
RatecodeID,29236,6,1,28373


In [109]:
X_train.RatecodeID.value_counts(normalize=False)

1     28373
2       676
5       106
3        65
4        15
99        1
Name: RatecodeID, dtype: int64

In [110]:
X_train.payment_type.value_counts(normalize=False)

1    19899
2     9142
3      144
4       51
Name: payment_type, dtype: int64

In [116]:
# need to reduce cardinality of PULocationID & DOLocationID
X_train.PULocationID.value_counts().sum

<bound method Series.sum of 237    1146
161    1088
236    1045
186    1026
162    1019
230    1000
234     924
170     923
142     905
48      892
79      828
138     814
163     787
239     724
132     717
107     714
68      704
164     678
141     622
238     605
100     600
249     589
229     558
90      521
231     517
263     498
140     474
113     469
264     440
148     422
       ... 
215       2
165       2
159       1
177       1
71        1
208       1
198       1
192       1
47        1
190       1
69        1
106       1
227       1
197       1
228       1
76        1
213       1
219       1
56        1
171       1
220       1
174       1
38        1
147       1
242       1
77        1
178       1
133       1
18        1
39        1
Name: PULocationID, Length: 151, dtype: int64>

In [113]:
import category_encoders as ce

encoder = ce.OneHotEncoder(use_cat_names=True)

X_train['RatecodeID'] = X_train['RatecodeID'].replace({'1': 'one', '2': 'two', '3': 'three', '4': 'four', '5': 'five', '99': 'six'})
X_test['RatecodeID'] = X_test['RatecodeID'].replace({'1': 'one', '2': 'two', '3': 'three', '4': 'four', '5': 'five', '99': 'six'})

X_train['store_and_fwd_flag'] = X_train['store_and_fwd_flag'].replace({'1': 'one', '2': 'two'})
X_test['store_and_fwd_flag'] = X_test['store_and_fwd_flag'].replace({'1': 'one', '2': 'two'})

X_train['payment_type'] = X_train['payment_type'].replace({'1': 'one', '2': 'two', '3': 'three', '4': 'four'})
X_test['payment_type'] = X_test['payment_type'].replace({'1': 'one', '2': 'two', '3': 'three', '4': 'four'})


X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.fit_transform(X_test)


X_train_encoded.head()

,Unnamed: 0,VendorID,passenger_count,trip_distance,RatecodeID_one,RatecodeID_two,RatecodeID_four,RatecodeID_three,RatecodeID_five,RatecodeID_six,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup,dropoff,trip_time
0,2466149,2,2,2.03,1,0,0,0,0,0,...,9.0,0.0,0.5,1.96,0.0,0.3,11.76,2017-03-06 12:10:08,2017-03-06 12:20:21,00:10:13
1,3406713,1,1,0.40,1,0,0,0,0,0,...,4.5,0.0,0.5,1.05,0.0,0.3,6.35,2017-06-14 08:11:30,2017-06-14 08:14:38,00:03:08
2,7071298,2,3,1.04,1,0,0,0,0,0,...,6.5,1.0,0.5,0.00,0.0,0.3,8.30,2017-11-22 19:42:47,2017-11-22 19:49:49,00:07:02
3,3845741,2,1,1.79,1,0,0,0,0,0,...,9.5,0.0,0.5,0.00,0.0,0.3,10.30,2017-11-12 16:58:55,2017-11-12 17:09:24,00:10:29
4,3672997,1,1,1.20,1,0,0,0,0,0,...,6.0,0.5,0.5,2.15,0.0,0.3,9.45,2017-06-14 22:01:39,2017-06-14 22:06:32,00:04:53


In [90]:
cat_vars = [var for var in X_train.columns if X_train[var].dtype == 'O']
cat_vars

['RatecodeID', 'store_and_fwd_flag', 'payment_type']

In [91]:
# this function will assign discrete values to the strings of the variables, 
# so that the smaller value corresponds to the smaller mean of target

def replace_categories(train, test, var, target):
    ordered_labels = train.groupby([var])[target].mean().sort_values().index
    ordinal_label = {k:i for i, k in enumerate(ordered_labels, 0)} 
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

for var in cat_vars:
    replace_categories(X_train, X_test, var, 'tip_amount')